In [1]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [6]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPool2D, Input
from tensorflow.keras.optimizers import SGD
from keras.optimizers import Adam
from tensorflow.keras.regularizers import L2
import warnings
from dataloader_v1 import SpectDataset
import numpy as np
from sklearn.metrics import classification_report
import itertools

In [7]:
warnings.filterwarnings('ignore')

In [8]:
def SBCNN_Model(input_shape, num_labels):
    model = Sequential()
    model.add(Input(input_shape))
    model.add(Conv2D(24, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='same'))
    model.add(MaxPool2D(strides=(4, 2)))
    model.add(Activation('relu'))

    model.add(Conv2D(48, 5, padding='valid'))
    model.add(Activation('relu'))

    model.add(Flatten())

    model.add(Dropout(0.5))
    model.add(Dense(64, kernel_regularizer=L2(0.001)))
    model.add(Activation('relu'))
    
    model.add(Dropout(0.5))
    model.add(Dense(num_labels, kernel_regularizer=L2(0.001)))
    model.add(Activation('softmax'))

    return model

In [9]:
def run_model(optimizer, n_iter):
    DATA_PATH = 'D:\\SP Cup 2022\\New folder\\spcup_2022_training_part1'
    Labeled_dir = 'D:\\SP Cup 2022\\New folder\\spcup_2022_training_part1\\labels.csv'

    dataloader = SpectDataset(add_noise=True, reverb=True)
    train_ds, val_ds = dataloader.call(DATA_PATH, Labeled_dir)

    for spectrogram, _ in train_ds.take(1):
        input_shape = spectrogram.shape
    model = SBCNN_Model(input_shape[1:], 6)

    model.compile(
        optimizer=optimizer,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
        metrics=['accuracy']
    )
    EPOCHS = n_iter
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,verbose=0
    # callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=2),
    )
    res_sparse = model.predict(val_ds)
    res_np = np.argmax(res_sparse, axis=1)
    pred_labels = list(res_np)
    true_labels = []
    for _, label in val_ds.take(1000):
        true_labels.extend(list(label.numpy()))

   
    print("SBCNN with optimizer {} and {} iterations, learning rate {}".format(optimizer, n_iter, model.optimizer._decayed_lr('float32').numpy()))
    class_names = [f'class_{i}' for i in range(6)]
    print(classification_report(true_labels, pred_labels, target_names=class_names))

In [10]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            initial_learning_rate=0.1,
            decay_steps=(1000), 
            decay_rate=0.9)

sgd = SGD(learning_rate = lr_schedule)
adm = Adam(learning_rate=lr_schedule)
optimizer = [sgd, adm]
n_iter = [10, 15]

params = list(itertools.product(optimizer, n_iter))

for p in params:
    run_model(p[0], p[1])

SBCNN with optimizer <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x0000019E2D56B2B0> and 10 iterations, learning rate 0.08538150042295456
              precision    recall  f1-score   support

     class_0       0.95      0.98      0.97       200
     class_1       0.81      0.74      0.77       200
     class_2       0.77      0.71      0.74       200
     class_3       0.91      0.95      0.93       200
     class_4       0.98      1.00      0.99       200
     class_5       0.69      0.72      0.71       200

    accuracy                           0.85      1200
   macro avg       0.85      0.85      0.85      1200
weighted avg       0.85      0.85      0.85      1200

SBCNN with optimizer <tensorflow.python.keras.optimizer_v2.gradient_descent.SGD object at 0x0000019E2D56B2B0> and 15 iterations, learning rate 0.06736113131046295
              precision    recall  f1-score   support

     class_0       0.98      0.98      0.98       200
     class_1       0.7